<a href="https://colab.research.google.com/github/adhyarya51/Python-setup/blob/main/Kisi_Kisi_LC3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perusahaan peminjaman sepeda di San Fransisco memintamu untuk menganalisa durasi perjalanan customer mereka. Untuk memudahkan analisa, lakukan hal-hal berikut ini:

## **STATISTIK DESKRIPTIF**

1. Lakukan perhitungan central tendency (mean, median, dan modus).
2. Lakukan pengecekan dispersi (standar deviasi, variance, dan range).
3. Lakukan pengecekan skewness dan kurtosis.
3. Analisa apakah ada data outlier berdasarkan sebaran datanya.
4. Cari tahu berapa persentase outlier terhadap data keseluruhan.
5. Bagaimana perubahan pemusatan data sebelum dan sesudah penghapusan outlier?

## **API**

Buatlah API untuk:
- `Menampilkan` seluruh entry data setelah dilakukan handling outlier.
- `Menghapus` entry data, dari data yang telah dihandling outlier.

clue:
- Load data csv yang sudah diolah dengan pandas, kemudian konversi data ke dictionary `df.to_dict()` atau json `df.to_json()` untuk dapat diolah lebih lanjut dengan API menggunakan FastAPI.
- Pembuatan API dilakukan pada script `.py`
- Endpoint dan penamaan function pada API dibebaskan.


---



Data dapat diakses dari BigQuery dengan ketentuan:

- Project_id: `bigquery-public-data`
- Dataset: `san_francisco_bikeshare`
- Table: `bikeshare_trips`
- Ambil hanya kolom `duration_sec` saja
- Berikan LIMIT pada data yang diambil sebesar 3000 entry data

Koneksikan data di atas menggunakan `BigQuery` ke Google Colab dengan code berikut:
```py
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

  project_id = "rock-wonder-317907" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)
```

Untuk melakukan Query menggunakan cara ini, kamu dapat menggunakan method `client.query('Masukkan Querynya').to_dataframe()`. Outputnya akan berupa Pandas dataframe, sehingga harus import Pandas.

Contoh:

```py
df = client.query('''
SELECT *
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE created_at < "2022-07-01"
ORDER BY year,month ASC
''').to_dataframe()
```



In [2]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "civil-clarity-417802" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


In [3]:
df = client.query('''
SELECT duration_sec
FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
limit 3000
''').to_dataframe()

In [5]:
df

,duration_sec
0,2189
1,865
2,488
3,501
4,471
...,...
2995,11013
2996,539
2997,857
2998,430


1. Lakukan perhitungan central tendency (mean, median, dan modus).

In [6]:
# Working area
print('mean duration sec adalah:',df['duration_sec'].mean())
print('median duration sec adalah:',df['duration_sec'].median())
print('mode duration sec adalah:',df['duration_sec'].mode().values[0])

mean duration sec adalah: 802.835
median duration sec adalah: 487.0
mode duration sec adalah: 354


2. Lakukan pengecekan dispersi (standar deviasi, variance, dan range).

In [7]:
std = df['duration_sec'].std()
print('Standard Deviation of duration_sec adalah:',std)

variance = df['duration_sec'].var()
print('Variance of duration_sec adalah:',variance)

range = df['duration_sec'].max() - df['duration_sec'].min()
print('Range duration_sec adalah:',range)

Standard Deviation of duration_sec adalah: 2871.1038086657622
Variance of duration_sec adalah: 8243237.080135046
Range duration_sec adalah: 84541


3. Lakukan pengecekan skewness dan kurtosis.

In [9]:
print(f"Skewness: {df['duration_sec'].skew()}")

print(f"Kurtosis: {df['duration_sec'].kurtosis()}")

Skewness: 19.615076327076103
Kurtosis: 443.306257431049


4. Analisa apakah ada data outlier berdasarkan sebaran datanya.

In [10]:
# menghitung Q1,Q2,Q3,IQR

q1 = df['duration_sec'].quantile(0.25)
q3 = df['duration_sec'].quantile(0.75)
iqr = q3-q1

up_bound = q3 + 3*iqr    #dikali 3 karena extreme biasanya kayak gini --> # up_bound = q3 + 1.5*iqr
low_bound = q1 - 3*iqr   #dikali 3 karena extreme biasanya kayak gini -->  # low_bound = q1 - 1.5*iqr

print('Q1:',q1)
print('Q3:',q3)
print('Interquartile Range (IQR):',iqr)
print(f'Upper Boundary: {up_bound:.2f}' )
print(f'Lower Boundary: {low_bound:.2f}')

Q1: 313.0
Q3: 770.0
Interquartile Range (IQR): 457.0
Upper Boundary: 2141.00
Lower Boundary: -1058.00


5. Cari tahu berapa persentase outlier terhadap data keseluruhan.

menggunakan skew method (distribusi tidak normal)

In [12]:
q1 = df['duration_sec'].quantile(0.25)
q3 = df['duration_sec'].quantile(0.75)
iqr = q3-q1

up_bound = q3 + 3*iqr
low_bound = q1 - 3*iqr

print(f'Upper Boundary: {up_bound:.2f}' )
print(f'Lower Boundary: {low_bound:.2f}')

Upper Boundary: 2141.00
Lower Boundary: -1058.00


In [13]:
# mencari outliers
outlier = df['duration_sec'][(df['duration_sec']<low_bound) | (df['duration_sec']>up_bound)]

# membuang outliers
no_outlier = df['duration_sec'][(df['duration_sec']>=low_bound) & (df['duration_sec']<=up_bound)]

# menghitung berapa persen outlier yg ada dari data keseluruhan
print(f'Percentage of outliers: {len(outlier)*100/len(df["duration_sec"]):.2f}%%')

Percentage of outliers: 2.60%%


6. Bagaimana perubahan pemusatan data sebelum dan sesudah penghapusan outlier?

In [16]:
df_cleaned = df[(df['duration_sec'] >= low_bound) & (df['duration_sec'] <= up_bound)]

mean_after = df_cleaned['duration_sec'].mean()
median_after = df_cleaned['duration_sec'].median()
mode_after = df_cleaned['duration_sec'].mode()


print("Mean Sebelum Penghapusan Outlier:", df['duration_sec'].mean())
print("Median Sebelum Penghapusan Outlier:", df['duration_sec'].median())
print("Mode Sebelum Penghapusan Outlier:", df['duration_sec'].mode())
print("Mean Setelah Penghapusan Outlier:", mean_after)
print("Median Setelah Penghapusan Outlier:", median_after)
print("Mode Setelah Penghapusan Outlier:", mode_after)

Mean Sebelum Penghapusan Outlier: 802.835
Median Sebelum Penghapusan Outlier: 487.0
Mode Sebelum Penghapusan Outlier: 0    354
Name: duration_sec, dtype: Int64
Mean Setelah Penghapusan Outlier: 572.1943874058863
Median Setelah Penghapusan Outlier: 475.0
Mode Setelah Penghapusan Outlier: 0    354
Name: duration_sec, dtype: Int64
